In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install sentence_transformers

In [ ]:
from torch.utils.data import DataLoader
#import math
from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
import logging
from datetime import datetime
import os
import csv

In [ ]:
train_df = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")
test_df = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
train_df.isnull().sum()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
y = train_df["id"]
features = ["lang_abv","language","label"]
X = train_df[features]
X

In [ ]:
label2int = {"contradiction": 0, "entailment": 1, "neutral":  2}
train_samples = []
dev_samples = []

In [ ]:
!pip install math

In [ ]:
import pandas as pd
import pandas.core as pc
import pandas as pd
import numpy as np
import argparse


In [ ]:
!pip install dateutil
!pip install pandas

In [ ]:
import math

In [ ]:
!pip install math

In [ ]:
%%capture
!pip install datasets==1.0.2
!pip install transformers

import datasets
import transformers

In [ ]:
T = train_df.shape

In [ ]:
from cs231n.classifiers import LinearSVM

In [ ]:
svm = LinearSVM()
tic = time.time()
loss_hist = svm.train(X_train, y_train, learning_rate=1e-7, reg=2.5e4,num_iters=1500, verbose=True)
toc = time.time()

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

inputs = tokenizer("/Dokumentumok/tok.txt", return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

inputs = tokenizer("/Dokumentumok/tok.txt", return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [ ]:
import numpy as np
import pandas as pd
import argparse
pred = pd.DataFrame()


input_value = train_df['premise']
imput_value2 = train_df['hypothesis']
label = train_df['label']
input_values= [input_value,imput_value2,label]
train_samples = []
dev_samples = []
train_data = train_df
train_data['label'] = train_data['label'].replace([0,2],[2,0])
for id, row in train_data.iterrows():
    label_id = int(row['label'])
    train_samples.append(input_values)
    
train_batch_size = 16
num_epochs = 4

#model = CrossEncoder('sentence_transformers/distilbert-base-nli-stsb-mean-tokens', num_labels=len(label2int))#
#model = CrossEncoder('sentence-transformers/distilbert-multilingual-nli-stsb-quora-ranking', num_labels=len(label2int))
#model = CrossEncoder('sentence-transformers/xlm-r-100langs-bert-base-nli-mean-tokens', num_labels=len(label2int))


model = CrossEncoder('joeddav/xlm-roberta-large-xnli', num_labels=len(label2int))
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)

evaluator = CESoftmaxAccuracyEvaluator.from_input_examples(dev_samples, name='ALLNLI-dev')

up_steps = math.ceil(len(train_dataloader)* num_epochs * 0.1)
logging.info(r"Warmup-steps: {}".format(up_steps))

#Train_the model
model.fit(train_dataloader=train_dataloader,
         evaluator=evaluator,
         epochs = num_epochs,
         evaluation_steps=10000,
         warmup_steps=up_steps)
        # output_path=model_save_path)

In [ ]:
!pip install pandas

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from transformers import BertTokenizer, BertForMultipleChoice
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMultipleChoice.from_pretrained('bert-base-uncased')

batch_size = 1
prompt = train_df.shape
choice0 = "premise"
choice1 = "hypothesis"
labels = torch.tensor(0).unsqueeze(0)  # choice0 is correct (according to Wikipedia ;)), batch size 1

encoding = tokenizer([[prompt, prompt], [choice0, choice1]], return_tensors='pt', padding=True)
outputs = model(**{k: v.unsqueeze(0) for k,v in encoding.items()}, labels=labels)  # batch size is 1

# the linear classifier still needs to be trained
loss = outputs.loss
logits = outputs.logits

In [ ]:
test_dataset = '../input/contradictory-my-dear-watson/test.csv'
df = pd.read_csv(test_dataset)
sentence_pairs =[]
ids = []
for id, row in df.iterrows():
    label_id = 0
    ids.append(row['id'])
    sentence_pairs.append([row['premise'], row['hypothesis']])
    
pred_scores = model.predict(sentence_pairs, convert_to_numpy=True, show_progress_bar=False, batch_size=4)
pred_labels = np.argmax(pred_scores, axis=1)
pred_labels


In [ ]:

out_df = pd.DataFrame([ids, pred_labels]).transpose()
out_df = out_df.rename(columns={0: 'id', 1: 'predictio'})
out_df ['prediction'] = out_df['prediction'].replace([2,0], [0,2])
out_df.to_csv('submission.csv',index=False)
